In [2]:
import pandas as pd
import pickle

Add rows for every talk that has multiple speakers
* Goal: have one row per speaker
* Enable search by speaker with occupation, about, and photo
* Good for SQL

How to do it?
* Find rows with multiple speakers
* Create new df of all speakers with talk_id
* **Need to somehow merge speakers and their occupations**
* Merge new df with old df
* Remove old row with multiple speakers

```python
ted_dict = {}

def get_speakers():
    for d in df['all_speakers']:
        for idx, spk in d.items():
            ted_dict[idx] = {'speaker': spk}
            
def get_occupations():
    for d in df['occupations']:
        for idx, occ in d.items():
            ted_dict[idx] = {'occupation': occ}
            
def get_about():
    for d in df['about_speakers']:
        for idx, txt in d.items():
            ted_dict[idx] = {'about_speaker': txt}
            
def get_photo():
    for d in df['photo_urls']:
        for idx, url in d.items():
            ted_dict[idx] = {'photo_url': url}
```

```python
ted_dict = {}
for row in df2.itertuples():
    # check each column to see if it has multiple speakers
    if len(row.all_speakers) > 1:
        row_dict = {}
        for idx, spk in row.all_speakers.items():
            row_dict[idx] = {'speaker': spk}
        for idx, occ in row.occupations.items():
            row_dict[idx].update({'occupation': occ})
        for idx, txt in row.about_speakers.items():
            row_dict[idx].update({'about_speaker': txt})
        for idx, url in row.photo_urls.items():
            row_dict[idx].update({'photo_url': url})
        for idx, _ in row_dict.items():
            row_dict[idx].update({'talk_id': row.talk_id})
```

In [3]:
# load pickle
with open('../data/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [4]:
df.shape

(4127, 20)

In [ ]:
mul = pd.read_csv('../data/multiple_speakers_talks.csv')

In [ ]:
mul.url.to_list()

In [ ]:
mul_df = df.loc[(df['url'].str.contains('alicia_garza')) |
                (df['url'].str.contains('adam_driver')) |
                (df['url'].str.contains('diana_reiss'))]

In [ ]:
mul_df

In [ ]:
def speaker_level_df(df):
    """Returns DataFrame with a row per each speaker."""
    dlist = []
    # get number of speakers
    for row in df.itertuples():
        num_spk = []
        num_spk.append(len(row.all_speakers))
        num_spk.append(len(row.occupations))
        num_spk.append(len(row.about_speakers))
        max_spk = max(num_spk)
        for i in range(max_spk):
            row_dict = {'talk_id': row.talk_id,
                        'speaker': row.all_speakers[i],
                        'occupation': row.occupations[i], 
                        'about_speaker': row.about_speakers[i],
                        'photo_url': row.photo_urls[i]}
            dlist.append(row_dict)
    return pd.DataFrame(dlist)

In [5]:
def speaker_level_df(df):
    """Returns DataFrame with a row per each speaker."""
    dlist = []
    # get number of speakers
    for row in df.itertuples():
        num_spk = []
        speakers = [None, None, None, None, None, None,]
        occupations = [None, None, None, None, None, None,]
        about_speakers = [None, None, None, None, None, None,]
        try:
            if isinstance(row.all_speakers, dict):
                num_spk.append(len(row.all_speakers))
                speakers = row.all_speakers
            if isinstance(row.occupations, dict):
                num_spk.append(len(row.occupations))
                occupations = row.occupations
            if isinstance(row.about_speakers, dict):
                num_spk.append(len(row.about_speakers))
                about_speakers = row.about_speakers
            else:
                num_spk = [1]
                speakers = [row.speaker_1]
        except:
            speakers = [row.speaker_1]
        max_spk = max(num_spk)
        for i in range(max_spk):
            try:
                if row.photo_urls[i] == '':
                    photo_urls = [None, None, None, None, None, None,]
                else:
                    photo_urls = row.photo_urls
                row_dict = {'talk_id': row.talk_id,
                            'speaker': speakers[i],
                            'occupation': occupations[i], 
                            'about_speaker': about_speakers[i],
                            'photo_url': photo_urls[i]}
                dlist.append(row_dict)
            except:
                row_dict = {'talk_id': row.talk_id,
                            'speaker': speakers[i],
                            'occupation': occupations[i], 
                            'about_speaker': about_speakers[i],
                            'photo_url': None}
                dlist.append(row_dict)
    return pd.DataFrame(dlist)

In [6]:
df.loc[df.talk_id == '2757', :]

,talk_id,title,speaker_1,all_speakers,occupations,about_speakers,views,recorded_date,published_date,event,native_lang,available_lang,comments,duration,topics,related_talks,url,photo_urls,description,transcript
2479,2757,How in vitro fertilization (IVF) works,Nassim Assefi,"{0: 'Nassim Assefi', 1: 'Brian A Levine'}","{0: ['doctor', 'novelist', 'guest host'], 1: [...",{0: 'Physician-novelist and activist writer Na...,1451748,2015-05-07,2017-09-07,TED-Ed,en,"[ar, el, en, es, fa, fr, he, it, ja, ko, pl, p...",None,402,"[TED-Ed, human body, pregnancy, children, biol...",{24178: 'How does the thyroid manage your meta...,https://www.ted.com/talks/nassim_assefi_and_br...,{0: ''},Infertility affects 1 in 8 couples worldwide. ...,"In 1978, Louise Brown became the world's first..."


In [37]:
# %debug
speaker_level_df(df.loc[df.talk_id == '2790', :])

,talk_id,speaker,occupation,about_speaker,photo_url
0,2790,Sofi Tukker,[band],None,https://pe.tedcdn.com/images/ted/3f0481fb497c1...


In [7]:
# %debug
spk_df = speaker_level_df(df)

In [54]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


In [8]:
spk_df.head()

,talk_id,speaker,occupation,about_speaker,photo_url
0,1,Al Gore,[climate advocate],Nobel Laureate Al Gore focused the world's att...,https://pe.tedcdn.com/images/ted/93b12c098c0aa...
1,92,Hans Rosling,[global health expert; data visionary],"In Hans Rosling’s hands, data sings. Global tr...",https://pe.tedcdn.com/images/ted/2bd4d0cfff0c3...
2,7,David Pogue,[technology columnist],David Pogue is the personal technology columni...,https://pe.tedcdn.com/images/ted/1437_253x190.jpg
3,53,Majora Carter,[activist for environmental justice],Majora Carter redefined the field of environme...,https://pe.tedcdn.com/images/ted/1320_254x191.jpg
4,66,Sir Ken Robinson,"[author, educator]",Creativity expert Sir Ken Robinson challenges ...,https://pe.tedcdn.com/images/ted/1465_254x191.jpg


In [9]:
spk_df.loc[(spk_df['occupation'].isna()) |
           (spk_df['about_speaker'].isna()) |
           (spk_df['photo_url'].isna())]

,talk_id,speaker,occupation,about_speaker,photo_url
297,353,David S. Rose,[angel investor],"""The Pitch Coach"" David S. Rose is an expert o...",None
1006,1244,Ian Ritchie,[software entrepreneur],None,https://pe.tedcdn.com/images/ted/0b117eb3cf69c...
1165,1407,Rick Falkvinge,None,"Rick Falkvinge, didn’t plan on becoming a poli...",https://pe.tedcdn.com/images/ted/d13dad96a8770...
1194,1436,Gary Kovacs,None,Gary Kovacs is a technologist and the former C...,https://pe.tedcdn.com/images/ted/eb8ff9cd54ca5...
1266,1515,Matt Mills,None,"Matt Mills comes from Aurasma, a startup that ...",https://pe.tedcdn.com/images/ted/c3cad40b2b629...
...,...,...,...,...,...
4273,64915,Susan Lupack,None,None,None
4277,64914,Alex Gendler,None,None,None
4279,65058,Philip Freeman,None,None,None
4281,65280,Iseult Gillespie,None,None,None


In [13]:
merged_df = pd.merge(spk_df, df, how='left', on='talk_id')

In [14]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4285 entries, 0 to 4284
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   talk_id         4285 non-null   object
 1   speaker         4285 non-null   object
 2   occupation      3715 non-null   object
 3   about_speaker   3745 non-null   object
 4   photo_url       3634 non-null   object
 5   title           4285 non-null   object
 6   speaker_1       4285 non-null   object
 7   all_speakers    4281 non-null   object
 8   occupations     3715 non-null   object
 9   about_speakers  3745 non-null   object
 10  views           4285 non-null   object
 11  recorded_date   4284 non-null   object
 12  published_date  4285 non-null   object
 13  event           4285 non-null   object
 14  native_lang     4285 non-null   object
 15  available_lang  4285 non-null   object
 16  comments        3572 non-null   object
 17  duration        4285 non-null   object
 18  topics  

In [16]:
drop_columns = merged_df.drop(columns=['occupations', 'about_speakers', 'photo_urls'])

In [19]:
drop_columns['is_main_speaker'] = (drop_columns['speaker'] == drop_columns['speaker_1'])

In [52]:
drop_columns['is_main_speaker'] = drop_columns['is_main_speaker'].map({True: 1, False: 0})

In [58]:
ted_df = drop_columns.copy()

In [63]:
ted_df.shape

(4285, 22)

In [64]:
ted_df.to_csv('../datasets/2020-07-29/ted_talks_en_spk_lvl.csv')